In [1]:
from pathlib import Path
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adset import AdSet
from facebook_business.adobjects.ad import Ad
from facebook_business.adobjects.campaign import Campaign
from facebook_business.adobjects.adcreative import AdCreative
from facebook_business.adobjects.targeting import Targeting
from facebook_business.api import FacebookRequest
import copy
from facebook_adapter import FacebookCampaignAdapter
# import facebook
import os

my_app_id = '958842090856883'
my_app_secret = 'a952f55afca38572cea2994d440d674b'
my_access_token = 'EAANoD9I4obMBAPcoZA5V7OZBQaPa3Tk7NMAT0ZBZCepdD8zZBcwMZBMHAM1zPeQiRY4Yw07rscee4LMRn9lMsJGuNZAYBA4nCYdZA6tsyL0KGTfQKIAFls3T5jul9Am6t95nbvcGXFmcFDYEyZAX2FpAuVesVGyiHuLFRKxlXfh5t6AZDZD'


In [2]:
import datetime
yesterday = datetime.datetime.now().date() - datetime.timedelta(1)

In [3]:
from facebook_datacollector import Campaigns
import mysql_adactivity_save
import pandas as pd
campaign_objective = {
    'LINK_CLICKS': 'link_click',
    'POST_ENGAGEMENT': 'post_engagement', 
    'VIDEO_VIEWS': 'video_view', 
    'CONVERSIONS':'offsite_conversion',
}

In [4]:
def copy_adset(adset_id):
    request = FacebookRequest(
            node_id=adset_id,
            method='POST',
            endpoint='/copies',
            api_type='EDGE',
            
    )
    params = {
            'deep_copy': True,
    }
    request.add_params(params)    
    response = request.execute()

    response_json = response.json()
#     print(response_json)
    
    new_adset_id = response_json.get('copied_adset_id')
#     print(new_adset_id)
    return new_adset_id


def modify_exists_adset(adset_id, adset_params):
    adset = AdSet(adset_id)
    
    for key in adset_params:
#         print(key, adset_params[key])
        adset[key] = adset_params[key]

    #print('[modify_exists_adset] adset:' , str(adset))
        
    update_response = adset.update()
    #print('update_response:' , update_response)
    
    remote_update_response = adset.remote_update()
    #print('remote_update_response:' , remote_update_response)
    
def config_adset_params_by_age(new_adset_id, age_max, age_min, init_bid=None):
    
    fields_adSet = [  AdSet.Field.campaign_id,  AdSet.Field.name, AdSet.Field.bid_amount, AdSet.Field.bid_strategy, AdSet.Field.daily_budget, AdSet.Field.budget_remaining, AdSet.Field.optimization_goal, AdSet.Field.bid_info, AdSet.Field.pacing_type
                    , AdSet.Field.attribution_spec, AdSet.Field.targeting]

    ad_set = AdSet(new_adset_id)
    ad_set_data = ad_set.remote_read(fields = fields_adSet)
    
    target = ad_set_data[AdSet.Field.targeting]
    original_name = ad_set_data[AdSet.Field.name]
                     
    target['age_max'] =  age_max
    target['age_min'] =  age_min

    if init_bid is None:
        adset_params = {
            AdSet.Field.name: original_name + ' ' +str(age_min) + '-' + str(age_max) ,
            AdSet.Field.targeting: target,
            'status': AdSet.Status.active,
        }
        ad_groups = ad_set.get_ads(fields=[
            AdSet.Field.name,
            AdSet.Field.campaign_id,
            AdSet.Field.configured_status,
        ])
    else:
        adset_params = {
            AdSet.Field.name: original_name + ' ' +str(age_min) + '-' + str(age_max) ,
            AdSet.Field.targeting: target,
            AdSet.Field.bid_amount: init_bid+1,
            'status': AdSet.Status.active,
        }
        ad_groups = ad_set.get_ads(fields=[
            AdSet.Field.name,
            AdSet.Field.campaign_id,
            AdSet.Field.bid_amount,
            AdSet.Field.configured_status,
        ])
    for ad in ad_groups:
        ad_id = ad['id']
#         print('ad_id' , ad_id)
        ad_object = Ad(ad_id)
        ad_object['status'] = Ad.Status.active
        ad_object.remote_update()
        
    return adset_params

def duplicate_asset_by_more_target(adset_id_which_want_copy, init_bid=None, bid_adjust=False):
    # duplicate more age target , use original age interval
    
    ad_set = AdSet(adset_id_which_want_copy)
    print(adset_id_which_want_copy)
    ad_set_data = ad_set.remote_read(fields = [AdSet.Field.targeting])
    target = ad_set_data[AdSet.Field.targeting]
    age_max = target['age_max']
    age_min = target['age_min']    
#     print('original age interval: ', age_min, '-' , age_max)
    
    interval_count = 2
    age_interval = int((age_max - age_min)/interval_count)
    
    current_adset_min = age_min
    current_adset_max = current_adset_min + age_interval
  
    for i in range(interval_count):
#         print(current_adset_min, current_adset_max)

        new_adset_id = copy_adset(adset_id_which_want_copy)
#         adset_params = config_adset_params_by_age(new_adset_id, current_adset_max, current_adset_min, init_bid)
        if bid_adjust is True and init_bid is not None:
            adset_params = config_adset_params_by_age(new_adset_id, current_adset_max, current_adset_min, init_bid)
        else:
            adset_params = config_adset_params_by_age(new_adset_id, current_adset_max, current_adset_min)
        modify_exists_adset(new_adset_id, adset_params)
        
        current_adset_min += age_interval
        current_adset_max += age_interval

In [ ]:
FacebookAdsApi.init(my_app_id, my_app_secret, my_access_token)
duplicate_asset_by_more_target(23843212213240457, 1, bid_adjust=False)

In [5]:
def check_if_campaign_day_target_reach(campaign):
    mydict = Campaigns(campaign).get_campaign_insights()
    charge_type = campaign_objective[Campaigns( campaign ).get_campaign_feature().get("target_type")]
    target_dict=dict()
    summary_dict=dict()
    for dic in mydict[0].get("actions"):
        if dic.get("action_type") == charge_type:
            summary_dict[campaign] = int( dic.get("value") )
    target = mysql_adactivity_save.get_campaign_target_dict()[campaign].iloc[0]
    campaign_days = Campaigns(campaign).get_campaign_feature()['campaign_days']
    charge_per_day = target/campaign_days
    target_dict[campaign] = charge_per_day
    if summary_dict[campaign] < target_dict[campaign]:
        return False
    else:
        return True

In [ ]:
FacebookAdsApi.init(my_app_id, my_app_secret, my_access_token)
campaign_list = mysql_adactivity_save.get_campaign()
for campaign in campaign_list:
    if check_if_campaign_day_target_reach(campaign) is False:
        fb = FacebookCampaignAdapter(campaign)
        fb.get_df()
        fb.get_bid()
        
        
        mydb=mysql_adactivity_save.connectDB("ad_activity")
        df = pd.read_sql("select * from adset_score where campaign_id=%s" %(campaign), con=mydb)
        df = df[df.request_time.dt.date==yesterday].sort_values(by=['score'], ascending=False)
        adset_id = df['adset_id'].iloc[0]
        init_bid = fb.init_bid_dict[ df['adset_id'].iloc[0] ]
        
        duplicate_asset_by_more_target(adset_id, init_bid, bid_adjust=True)

In [6]:
def duplicate_highest_ranking_adset_with_higher_bid():
    campaign_list = mysql_adactivity_save.get_campaign()
    for campaign in campaign_list:
        if check_if_campaign_day_target_reach(campaign) is False:
            fb = FacebookCampaignAdapter(campaign)
            fb.get_df()
            fb.get_bid()

            mydb=mysql_adactivity_save.connectDB("ad_activity")
            df = pd.read_sql("select * from adset_score where campaign_id=%s" %(campaign), con=mydb)
            df = df[df.request_time.dt.date==yesterday].sort_values(by=['score'], ascending=False)
            adset_id = df['adset_id'].iloc[0]
            init_bid = fb.init_bid_dict[ df['adset_id'].iloc[0] ]

            duplicate_asset_by_more_target(adset_id, init_bid, bid_adjust=True)

In [7]:
FacebookAdsApi.init(my_app_id, my_app_secret, my_access_token)
duplicate_highest_ranking_adset_with_higher_bid()

23843212221300457
25 34
ad_id 23843220010850457
ad_id 23843220010820457
True
34 43
ad_id 23843220011670457
ad_id 23843220011610457
True


In [11]:
def duplicate_highest_ranking_adset():
    campaign_list = mysql_adactivity_save.get_campaign()
    for campaign in campaign_list:
        if check_if_campaign_day_target_reach(campaign) is False:
            mydb=mysql_adactivity_save.connectDB("ad_activity")
            df = pd.read_sql("select * from adset_score where campaign_id=%s" %(campaign), con=mydb)
            df = df[df.request_time.dt.date==yesterday].sort_values(by=['score'], ascending=False)
            adset_id = df['adset_id'].iloc[0]
            duplicate_asset_by_more_target(adset_id, bid_adjust=False)

In [12]:
duplicate_highest_ranking_adset()

23843212221300457
25 34
ad_id 23843220016190457
ad_id 23843220016150457
False
34 43
ad_id 23843220016730457
ad_id 23843220016680457
False
